In [1]:
!pip install rouge
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [43]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re

from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm

In [44]:
model_checkpoints = "alaggung/bart-pretrained"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 

In [45]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (..) 제거
    #sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    # 스폐셜 토큰 적용할 거면 여기 위에 영어 외 문자 공백으로 만들때 스폐셜 토큰을 넘어갈 수 있도록 지정해주면된다.
    # 그리고 세번째 정규표현식을 지워야 할 것이다. 
    return sentence

In [46]:
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [47]:
train_Category = train_df['Category'].unique()
val_Category = val_df['Category'].unique()
def categori_ext(data, Category, tv):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
    return df

In [48]:
train_df = categori_ext(train_df, train_Category, 'train')
val_df = categori_ext(val_df, val_Category, 'val')

In [49]:
train_df.head()

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)


In [50]:
# 전체 Text 데이터에 대한 전처리 (1)
train_text = []
val_text = []

for tt in tqdm(train_df['Text']):
    train_text.append(preprocess_sentence(tt))

for vt in tqdm(val_df['Text']):
    val_text.append(preprocess_sentence(vt))

100%|██████████| 1746/1746 [00:00<00:00, 32178.67it/s]


In [51]:
train_df = pd.DataFrame(zip(train_text), columns=['Text'])
val_df = pd.DataFrame(zip(val_text), columns=['Text'])

In [52]:
train_df.head()

,Text
0,그럼 날짜는 가격 큰 변동 없으면 6 28 7 13로 확정할까 우리 비행포함 15일...
1,kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마스크말고 부직포 마...
2,아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애 그러...
3,칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남 아 그 왕칫솔 또 사려나 싶...
4,잠도안오네 얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보 빼이보릿 되버렸네...


In [53]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)

In [54]:
print(train_data)
print(val_data)

Dataset(features: {'Text': Value(dtype='string', id=None)}, num_rows: 13994)
Dataset(features: {'Text': Value(dtype='string', id=None)}, num_rows: 1746)


In [55]:
max_input = 128
max_target = 128
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [56]:
 def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [57]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [58]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Text'])):
        label_id.append(tokenizer.encode(data_to_process['Text'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Text'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Text'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [59]:
special_words = [
                "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
                "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
                "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
                "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
                ]

tokenizer.add_special_tokens({"additional_special_tokens": special_words})
model.resize_token_embeddings(len(tokenizer))

model.config.max_length = 128 
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5



In [20]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [60]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text'])

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [61]:
model.config

BartConfig {
  "_name_or_path": "alaggung/bart-pretrained",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 2,
  "classifier_dropout": 0.1,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 3,
  "forced_eos_token_id": 3,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "length_penalty": 2.0,
  "max_length": 128,
  "max_position_embeddings": 512,
  "model_type": "bart",
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "num_hidden_layers":

In [62]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)
   

In [63]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/bartpretrained_MLM_test",
    num_train_epochs=10, # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=500,
    save_total_limit=3,
 


    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [64]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [65]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [66]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13994
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8750
  Number of trainable parameters = 46714880
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,6.221100
1000,3.340900
1500,3.204900
2000,3.147700
2500,3.134200
3000,3.107500
3500,3.094400
4000,3.072200
4500,3.063100
5000,3.053400


Saving model checkpoint to checkpoint/bartpretrained_MLM_test/checkpoint-500
Configuration saved in checkpoint/bartpretrained_MLM_test/checkpoint-500/config.json
Model weights saved in checkpoint/bartpretrained_MLM_test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in checkpoint/bartpretrained_MLM_test/checkpoint-500/tokenizer_config.json
Special tokens file saved in checkpoint/bartpretrained_MLM_test/checkpoint-500/special_tokens_map.json
Saving model checkpoint to checkpoint/bartpretrained_MLM_test/checkpoint-1000
Configuration saved in checkpoint/bartpretrained_MLM_test/checkpoint-1000/config.json
Model weights saved in checkpoint/bartpretrained_MLM_test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in checkpoint/bartpretrained_MLM_test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in checkpoint/bartpretrained_MLM_test/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to checkpoint/bartpretrained_MLM_test/checkpoint-1500

TrainOutput(global_step=8750, training_loss=3.2622414132254463, metrics={'train_runtime': 3181.2787, 'train_samples_per_second': 43.989, 'train_steps_per_second': 2.75, 'total_flos': 4743957294612480.0, 'train_loss': 3.2622414132254463, 'epoch': 10.0})